## Using the `TSP` Action in the `OPTNETWORK` Actionset To Find the Shortest Tour Between the US Busiest Airports

------------------------------------------------------------------------------------------------------------------------------

In this Jupyter notebook, we provide an application of the Travel Salesman Problem to find the shortest tour that visits 60 busiest US airports. 
We start by building a network of major US airports as the nodes and the existing air routes between airports as our directed links. The required datasets, including airports, airlines, and routes, are obtained from the "openflights.org" website. 
A feasible tour is a cycle visiting every vertex exactly once, and it is called a Hamiltonian cycle. Note that determining whether or not such a tour exists is an NP-hard problem. 
The code for reading the data as well as the data preparation and cleaning steps, are stored in the "helper.py" script. 

To read more about the `TSP` algorithm, refer to [SAS Documentation](https://pubshelpcenter.unx.sas.com/test/doc/en/pgmsascdc/default/casactnopt/cas-optnetwork-tsp.htm?fromDefault=).  

------------------------------------------------------------------------------------------------------------------------------
The flow of this notebook is as follows:
* [Reading the airports, airlines, and routes data from openflights website](#1st-bullet)
* [Connecting to CAS server](#2nd-bullet)
* [Uploading links (routes) and nodes(airports) DataFrame to CAS ](#3rd-bullet)
* [Calling the `TSP` action](#7th-bullet)
* [Visualizing the result](#8th-bullet)

------------------------------------------------------------------------------------------------------------------------------
**Prepared by:** Hossein Tohidi (<i class="fa fa-github" aria-hidden="true"></i> [Hossein-Tohidi](https://github.com/Hossein-Tohidi))


In [1]:
import swat
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
%matplotlib notebook  
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),r"../../common/python"))
import cas_connection as cas 
import geopandas as gpd
import networkx as nx
from shapely.geometry import Polygon
import helper
import warnings
warnings.filterwarnings("ignore",category=FutureWarning) 

### Reading the airports, airlines, and routes data from openflights website <a class="anchor" id="1st-bullet"></a>

In [19]:
airports = helper.read_airports("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat")
airports.head()

AirportID                                         Name          City  \
0          1                               Goroka Airport        Goroka   
1          2                               Madang Airport        Madang   
2          3                 Mount Hagen Kagamuga Airport   Mount Hagen   
3          4                               Nadzab Airport        Nadzab   
4          5  Port Moresby Jacksons International Airport  Port Moresby   

            Country IATA  ICAO  Latitude   Longitude  Altitude Timezone DST  \
0  Papua New Guinea  GKA  AYGA -6.081690  145.391998      5282       10   U   
1  Papua New Guinea  MAG  AYMD -5.207080  145.789001        20       10   U   
2  Papua New Guinea  HGU  AYMH -5.826790  144.296005      5388       10   U   
3  Papua New Guinea  LAE  AYNZ -6.569803  146.725977       239       10   U   
4  Papua New Guinea  POM  AYPY -9.443380  147.220001       146       10   U   

                     Tz     Type       Source node  \
0  Pacific/Port_Moresby  airport  OurAirports  GKA   
1  Pacific/Port_Moresby  airport  OurAirports  MAG   
2  Pacific/Port_Moresby  airport  OurAirports  HGU   
3  Pacific/Port_Moresby  airport  OurAirports  LAE   
4  Pacific/Port_Moresby  airport  OurAirports  POM   

                                        pos  
0       (145.391998291, -6.081689834590001)  
1           (145.789001465, -5.20707988739)  
2  (144.29600524902344, -5.826789855957031)  
3                   (146.725977, -6.569803)  
4   (147.22000122070312, -9.44338035583496)

### Find Optimal Tour to Visit US Busiest Airports

In [3]:
major_airports = ['ABQ', 'ATL', 'AUS', 'BDL', 'BNA', 'BOI', 'BOS', 'BUR', 'BWI', 'CHS',
                  'CLE', 'CLT', 'CMH', 'CVG', 'DAL', 'DCA', 'DEN', 'DFW', 'DTW', 'EWR',
                  'FLL', 'HOU', 'IAD', 'IAH', 'IND', 'JAX', 'JFK', 'LAS', 'LAX', 'LGA',
                  'MCI', 'MCO', 'MDW', 'MEM', 'MIA', 'MKE', 'MSP', 'MSY', 'OAK', 'OMA',
                  'ONT', 'ORD', 'PBI', 'PDX', 'PHL', 'PHX', 'PIT', 'RDU', 'RNO', 'RSW',
                  'SAN', 'SAT', 'SEA', 'SFO', 'SJC', 'SLC', 'SMF', 'SNA', 'STL', 'TPA']
airports = airports[airports['IATA'].isin(major_airports)].reset_index(drop=True)
airports.head()

AirportID                                               Name         City  \
0       3448  General Edward Lawrence Logan International Ai...       Boston   
1       3453         Metropolitan Oakland International Airport      Oakland   
2       3454                                    Eppley Airfield        Omaha   
3       3458                  Kansas City International Airport  Kansas City   
4       3462           Phoenix Sky Harbor International Airport      Phoenix   

         Country IATA  ICAO   Latitude   Longitude  Altitude Timezone DST  \
0  United States  BOS  KBOS  42.364300  -71.005203        20       -5   A   
1  United States  OAK  KOAK  37.721298 -122.221001         9       -8   A   
2  United States  OMA  KOMA  41.303200  -95.894096       984       -6   A   
3  United States  MCI  KMCI  39.297600  -94.713898      1026       -6   A   
4  United States  PHX  KPHX  33.434299 -112.012001      1135       -7   N   

                    Tz     Type       Source node  \
0     America/New_York  airport  OurAirports  BOS   
1  America/Los_Angeles  airport  OurAirports  OAK   
2      America/Chicago  airport  OurAirports  OMA   
3      America/Chicago  airport  OurAirports  MCI   
4      America/Phoenix  airport  OurAirports  PHX   

                                        pos  
0               (-71.00520325, 42.36429977)  
1                  (-122.221001, 37.721298)  
2                     (-95.894096, 41.3032)  
3                     (-94.713898, 39.2976)  
4  (-112.01200103759766, 33.43429946899414)

In [4]:
airlines = helper.read_airlines("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat")
airlines.head()

Airline ID          Airline Name Alias Airline IATA ICAO  Callsign  \
1            1        Private flight    \N            -  NaN       NaN   
3            3         1Time Airline    \N           1T  RNX   NEXTIME   
10          10           40-Mile Air    \N           Q5  MLA  MILE-AIR   
13          13      Ansett Australia    \N           AN  AAA    ANSETT   
14          14  Abacus International    \N           1B  NaN       NaN   

   Airline Country Active  
1              NaN      Y  
3     South Africa      Y  
10   United States      Y  
13       Australia      Y  
14       Singapore      Y

In [5]:
routes = helper.read_routes("https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat",airports,airlines)
# only routes from/to major airports
routes = routes[(routes['from'].isin(major_airports))&(routes['to'].isin(major_airports))]
# let's remove multiple links in routes and drop unneccessary columns
routes = routes[['from','to']].drop_duplicates(keep='first')
routes.head()

from   to
0  FLL  MCO
1  FLL  TPA
2  JAX  TPA
3  MCO  FLL
4  MCO  RSW

In [6]:
# join airports Longitude-Latitude to route
routes = routes.merge(airports[['IATA','Longitude','Latitude']], how='left', left_on='from',right_on='IATA').drop('IATA',axis=1)
routes.rename(columns = {'Longitude':'fromLon','Latitude':'fromLat'},inplace=True)
routes = routes.merge(airports[['IATA','Longitude','Latitude']], how='left', left_on='to',right_on='IATA').drop('IATA',axis=1)
routes.rename(columns = {'Longitude':'toLon','Latitude':'toLat'},inplace=True)
routes.head()

from   to    fromLon    fromLat      toLon      toLat
0  FLL  MCO -80.152702  26.072599 -81.308998  28.429399
1  FLL  TPA -80.152702  26.072599 -82.533203  27.975500
2  JAX  TPA -81.687897  30.494101 -82.533203  27.975500
3  MCO  FLL -81.308998  28.429399 -80.152702  26.072599
4  MCO  RSW -81.308998  28.429399 -81.755203  26.536200

In [7]:
# calculate distance for each edge
routes['weight'] = routes.apply(lambda x:
                                helper.find_great_circle_distance(x["fromLon"],x["fromLat"],x["toLon"],x["toLat"]), axis=1)
routes.head()

from   to    fromLon    fromLat      toLon      toLat      weight
0  FLL  MCO -80.152702  26.072599 -81.308998  28.429399  285.991147
1  FLL  TPA -80.152702  26.072599 -82.533203  27.975500  316.898801
2  JAX  TPA -81.687897  30.494101 -82.533203  27.975500  291.907495
3  MCO  FLL -81.308998  28.429399 -80.152702  26.072599  285.991147
4  MCO  RSW -81.308998  28.429399 -81.755203  26.536200  215.133498

### Connecting to CAS server <a class="anchor" id="2nd-bullet"></a>

In [8]:
s = cas.reconnect()
s.loadActionSet('network')
s.loadActionSet('optnetwork')

NOTE: Added action set 'network'.
NOTE: Added action set 'optnetwork'.


[actionset]

 'optnetwork'

+ Elapsed: 0.00221s, user: 0.000547s, sys: 0.00165s, mem: 0.221mb

### Uploading links (routes) and nodes(airports) DataFrame to CAS <a class="anchor" id="3rd-bullet"></a>

In [9]:
s.upload(routes, casout = {'name':'routes_links', 'replace':True})
s.upload(airports, casout = {'name':'aiports_nodes', 'replace':True})

### Calling the `TSP` action and check the status <a class="anchor" id="4th-bullet"></a>

In [10]:
result = s.optNetwork.tsp(
            links       = {"name":"routes_links"},
            direction   = 'directed',
            out         = {"name":"TSPTour", "replace":True})

NOTE: The number of nodes in the input graph is 60.
NOTE: The number of links in the input graph is 2134.
NOTE: The TSP solver is starting using an augmented symmetric graph with 120 nodes and 2194 links.
NOTE: Processing the traveling salesman problem using 1 threads across 1 machines.
NOTE: The initial TSP heuristics found a tour with cost 31720.648582 using 0.03 (cpu: 0.03) seconds.
NOTE: The MILP presolver value NONE is applied.
NOTE: The MILP solver is called.
NOTE: The Branch and Cut algorithm is used.
NOTE: The MILP solver added 26 cuts with 2126 cut coefficients at the root.
NOTE: Optimal.
NOTE: Objective = 25278.638868.
NOTE: Processing the traveling salesman problem used 0.12 (cpu: 0.12) seconds.


In [11]:
tourDF =s.CASTable('TSPTour').to_frame()
tourDF.head()

Selected Rows from Table TSPTOUR

   tsp_order from   to      weight
0        0.0  ABQ  PHX  527.877675
1        1.0  PHX  SAN  488.768924
2        2.0  SAN  LAX  175.756439
3        3.0  LAX  SJC  495.885628
4        4.0  SJC  BUR  476.850625

In [12]:
airports_merged = airports.merge(tourDF[['from','tsp_order']],how='left',left_on='IATA',right_on='from').drop('from',axis=1)
airports_merged = airports_merged[['node','tsp_order','Name','IATA','City','Country','Latitude','Longitude']].sort_values('tsp_order')
airports_merged.head()

node  tsp_order                                             Name IATA  \
59  ABQ        0.0                Albuquerque International Sunport  ABQ   
4   PHX        1.0         Phoenix Sky Harbor International Airport  PHX   
38  SAN        2.0                  San Diego International Airport  SAN   
7   LAX        3.0                Los Angeles International Airport  LAX   
41  SJC        4.0  Norman Y. Mineta San Jose International Airport  SJC   

           City        Country   Latitude   Longitude  
59  Albuquerque  United States  35.040199 -106.609001  
4       Phoenix  United States  33.434299 -112.012001  
38    San Diego  United States  32.733601 -117.190002  
7   Los Angeles  United States  33.942501 -118.407997  
41     San Jose  United States  37.362598 -121.929001

### Visualizing the result <a class="anchor" id="8th-bullet"></a>

In [13]:
G = nx.from_pandas_edgelist(tourDF, source='from', target='to',create_using=nx.DiGraph())
nodesDF_filtered = airports_merged[['node','Latitude','Longitude','IATA']].drop_duplicates(keep='first')
nodesDF_filtered['pos'] = list(zip(nodesDF_filtered['Longitude'], nodesDF_filtered['Latitude']))
attrs = nodesDF_filtered.set_index('node').to_dict('index')
nx.set_node_attributes(G, attrs)
pos = nx.get_node_attributes(G, 'pos')
fig, ax = plt.subplots(figsize=(12,8))
countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
mplg = countries.loc[countries["name"] == "United States of America",'geometry'] #MULTIPOLYGON 
mplg.intersection(Polygon([[-125,20],[-125,50],[-60,50],[-60,20]])).plot(color="lightgrey",ax=ax)
labels = nx.get_node_attributes(G, 'IATA')
labels_pos = {key:(val[0],val[1]+0.5) for key,val in pos.items()}
labels_pos = helper.modify_labels_pos_randomly(labels_pos,threshold=0.7)
nx.draw_networkx_labels(G,labels_pos, font_size=5, labels=labels)
nx.draw_networkx_nodes(G, pos,node_size=20, node_shape='d',ax=ax,node_color='red',alpha=0.8)
for edge in G.edges(data=True):
    nx.draw_networkx_edges(G, pos,
                           edgelist=[(edge[0],edge[1])],arrows=True, 
                           connectionstyle=f'arc3, rad = {0.1}', width=0.7,
                           node_size=20,
                           arrowsize=5,
                           ax=ax)
plt.show()

<IPython.core.display.Javascript object>